In [1]:
import matplotlib.pyplot as plt
import xarray as xr
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from statsmodels.stats import multitest as smm
from matplotlib.ticker import ScalarFormatter
from detclim.results_plot import fmt_case

plt.style.use("default")

In [ ]:
ALPHA = 0.05
_THR = {
    0.01: 7,
    0.05: 11
}

ctl_key = tuple(["PertLim 1.0e-10"] * 2)
run_len = "1year"
rolling = 12
niter = 1000
REJECT_UC_THR = _THR[ALPHA]

cases = [
    ("ctl", "ctl"),
    # ("ctl", "ctl-next"),
    ("ctl", "effgw_oro-0p5pct"),
    ("ctl", "effgw_oro-1p0pct"),
    ("ctl", "effgw_oro-5p0pct"),
    ("ctl", "effgw_oro-10p0pct"),
    ("ctl", "effgw_oro-20p0pct"),
    ("ctl", "effgw_oro-30p0pct"),
    ("ctl", "effgw_oro-40p0pct"),
    ("ctl", "effgw_oro-50p0pct"),
    ("ctl", "clubb_c1-1p0pct"),
    ("ctl", "clubb_c1-3p0pct"),
    ("ctl", "clubb_c1-5p0pct"),
    ("ctl", "clubb_c1-10p0pct"),
]
# cases = [
#     ("ctl", "ctl"),
#     ("ctl-v3", "ctl-v3"),
#     ("effgw_oro-0p5pct", "effgw_oro-0p5pct"),
#     ("effgw_oro-1p0pct", "effgw_oro-1p0pct"),
#     ("effgw_oro-10p0pct", "effgw_oro-10p0pct"),
#     ("effgw_oro-20p0pct", "effgw_oro-20p0pct"),
#     ("effgw_oro-30p0pct", "effgw_oro-30p0pct"),
#     ("effgw_oro-40p0pct", "effgw_oro-40p0pct"),
#     ("effgw_oro-50p0pct", "effgw_oro-50p0pct"),
# ]
pcts = [0.5, 1, 5, 10, 20, 30, 40, 50]
# pcts = [0.5, 1]
files = [
    Path(
        "bootstrap_data/bootstrap_output.{}_{}avg_ts40.{}_{}_n{}.nc".format(
            run_len, rolling, *case, niter
        )
    )
    for case in cases
]

# files.append(
#     Path(
#         "bootstrap_data/bootstrap_output.1year_12avg_ts40.ctl_effgw_oro-0p5pct_n1000.nc"
#     )
# )
# cases.append(("ctl_ts40", "effgw_oro-0p5pct_ts40"))
print("[")
for _file in files:
    print(f"\t{_file.name:>60s}\t\t{_file.exists()}")
print("]")


In [ ]:
fig_dpi = 300
fig_width = 12.5 / 2.54
fig_height = fig_width / 3.75
fig_extn = "pdf"

In [ ]:
ks_pval_cr = {}
ks_pval_cr_i = {}

n_reject = {}
reject_test = {}

n_reject_cr = {}
reject_test_cr = {}

n_reject_cr_i = {}
reject_test_cr_i = {}

for _ix, _file in enumerate(files):
    case_a, case_b = cases[_ix]
    n_iter = int(_file.stem.split("_")[-1][1:])
    case_a = fmt_case(case_a)
    case_b = fmt_case(case_b)
    ks_res = xr.open_dataset(_file)
    ks_pval = ks_res["pval"].values

    fig, axes = plt.subplots(1, 2, figsize=(15, 4))
    quantile = ALPHA * 100
    time_step = np.arange(ks_res.time.shape[0])

    # n_reject = np.array((ks_pval < ALPHA).sum(axis=1))
    n_reject[(case_a, case_b)] = np.array((ks_pval < ALPHA).sum(axis=1))

    n_reject_mean = np.median(n_reject[(case_a, case_b)], axis=0)
    n_reject_lq = np.percentile(n_reject[(case_a, case_b)], quantile, axis=0)
    n_reject_uq = np.percentile(n_reject[(case_a, case_b)], 100 - quantile, axis=0)

    _pval_cr = []
    _pval_cr_i = []

    for jdx in range(ks_pval.shape[0]):
        _pval_cr.append(
            smm.fdrcorrection(
                ks_pval[jdx].flatten(),
                alpha=ALPHA,
                method="n",
                is_sorted=False,
            )[1].reshape(ks_pval[jdx].shape)
        )

        _pval_cr_i.append(
            smm.fdrcorrection(
                ks_pval[jdx].flatten(),
                alpha=ALPHA,
                method="i",
                is_sorted=False,
            )[1].reshape(ks_pval[jdx].shape)
        )
    ks_pval_cr[(case_a, case_b)] = np.array(_pval_cr)
    ks_pval_cr_i[(case_a, case_b)] = np.array(_pval_cr_i)

    n_reject_cr[(case_a, case_b)] = np.array(
        (ks_pval_cr[(case_a, case_b)] < ALPHA).sum(axis=1)
    )
    n_reject_cr_i[(case_a, case_b)] = np.array(
        (ks_pval_cr_i[(case_a, case_b)] < ALPHA).sum(axis=1)
    )

    n_reject_mean_cr = np.median(n_reject_cr[(case_a, case_b)], axis=0)
    n_reject_lq_cr = np.percentile(n_reject_cr[(case_a, case_b)], quantile, axis=0)
    n_reject_uq_cr = np.percentile(
        n_reject_cr[(case_a, case_b)], 100 - quantile, axis=0
    )

    rejections = {
        f"{100 * (1 - ALPHA)}%": n_reject_uq,
        f"{100 * (1 - ALPHA)}% [Corrected]": n_reject_uq_cr,  # .max(axis=0),
    }
    width = 0.5
    mult = 0
    for name, nreject in rejections.items():
        offset = width * mult
        rect = axes[0].bar(time_step + offset, nreject, width=0.45, label=name)
        axes[0].bar_label(rect, padding=3, color=rect[-1].get_facecolor())
        mult += 1

    axes[0].bar_label(rect, padding=3)
    # try:
    #     axes[0].set_xticks(time_step, ["1-12", "2-REJECT_UC_THR", "3-14"])
    # except ValueError:

    # axes[0].axhline(
    #     ALPHA * ks_pval.shape[1],
    #     color="k",
    #     ls="-",
    #     label=f"{ALPHA * 100}% of variables",
    #     zorder=1,
    # )
    axes[0].axhline(
        REJECT_UC_THR,
        color="#600",
        ls="--",
        label=f"Failure thr [uncorrected]",
        zorder=1,
    )
    axes[0].axhline(
        1,
        color="#060",
        ls="-.",
        label=f"Failure thr [corrected]",
        zorder=1,
    )
    axes[0].legend()

    axes[0].set_title(
        f"Number of variables rejected at {(1 - ALPHA) * 100}% confidence"
    )
    axes[0].set_xlabel("Simulated month")
    axes[0].set_ylabel("N variables")

    test = {
        "Un-corrected": (n_reject[(case_a, case_b)] > REJECT_UC_THR).sum(axis=0),
        # "Corrected": (n_reject_cr > ks_pval.shape[1] * ALPHA).sum(axis=0),
        "Corrected": (n_reject_cr[(case_a, case_b)] > 0).sum(axis=0),
    }

    mult = 0
    for name, itest in test.items():
        offset = width * mult
        rect = axes[1].bar(time_step + offset, itest, width=0.45, label=name)
        axes[1].bar_label(rect, padding=3, color=rect[-1].get_facecolor(), zorder=10)
        mult += 1

    axes[1].axhline(
        ALPHA * ks_pval.shape[0],
        color="#343",
        ls="-.",
        label=f"{ALPHA * 100}% of tests",
        zorder=1,
    )
    axes[1].legend()

    for _ax in axes:
        _ax.set_xticks(
            time_step, [f"{xi}-{xi + 11}" for xi in range(1, len(time_step) + 1)]
        )

    axes[1].set_xlabel("Simulated month")

    axes[1].set_title(f'Number of tests (of {ks_pval.shape[0]}) "failing"')

    _reject = f"{ALPHA:.2f}".replace(".", "p")
    fig.suptitle(f"{case_a} x {case_b}")
    plt.tight_layout()
    plt.savefig(f"plt_{case_a}-{case_b}_n{n_iter}.png")

fig, axes = plt.subplots(1, 1, figsize=(12, 6))
quantile = ALPHA * 100
bar_width = 0.3  # the width of the bars

for idx, _file in enumerate(files):
    # print(_file.stem.split("_"))
    # case_a, case_b = _file.stem.split("_")[2:4]
    case_a, case_b = cases[idx]
    n_iter = int(_file.stem.split("_")[-1][1:])
    case_a = fmt_case(case_a)
    case_b = fmt_case(case_b)
    ks_res = xr.open_dataset(_file)
    ks_pval = ks_res["pval"].values

    time_step = np.arange(ks_res.time.shape[0])

    n_reject[(case_a, case_b)] = np.array((ks_pval < ALPHA).sum(axis=1))
    n_reject_mean = np.median(n_reject[(case_a, case_b)], axis=0)
    n_reject_lq = np.percentile(n_reject[(case_a, case_b)], quantile, axis=0)
    n_reject_uq = np.percentile(n_reject[(case_a, case_b)], 100 - quantile, axis=0)

    _pval_cr = []
    _pval_cr_indp = []
    # print(f"APPLY FDR CORRECTION FOR {ks_pval.shape[0]} iters")
    for jdx in range(ks_pval.shape[0]):
        _pval_cr.append(
            smm.fdrcorrection(
                ks_pval[jdx].flatten(),
                alpha=ALPHA,
                method="n",
                is_sorted=False,
            )[1].reshape(ks_pval[jdx].shape)
        )
        _pval_cr_indp.append(
            smm.fdrcorrection(
                ks_pval[jdx].flatten(),
                alpha=ALPHA,
                method="i",
                is_sorted=False,
            )[1].reshape(ks_pval[jdx].shape)
        )
        
    ks_pval_cr = np.array(_pval_cr)
    ks_pval_cr_i = np.array(_pval_cr_indp)
    
    # n_reject_cr[(case_a, case_b)] = np.array((ks_pval_cr < ALPHA).sum(axis=1))
    n_reject_mean_cr = np.median(n_reject_cr[(case_a, case_b)], axis=0)
    n_reject_lq_cr = np.percentile(n_reject_cr[(case_a, case_b)], quantile, axis=0)
    n_reject_uq_cr = np.percentile(
        n_reject_cr[(case_a, case_b)], 100 - quantile, axis=0
    )

    reject_test[(case_a, case_b)] = n_reject_uq
    reject_test_cr[(case_a, case_b)] = n_reject_uq_cr

    # ln_uq, = axes.bar(
    #     time_step, n_reject_uq, color=f"C{idx}", lw=1.5, label=f"{case_a} x {case_b}"
    # ),
    # ln_uq_cr, = axes.bar(
    #     time_step,
    #     n_reject_uq_cr,
    #     color=f"C{idx}",
    #     lw=1.5,
    #     ls="--",
    #     label=f"{case_a} x {case_b} [corr]"
    # ),

    offset = bar_width * idx

    if False:
        (ln_uq,) = (
            axes.bar(
                time_step - offset / 2,
                n_reject_uq,
                width=width / 2,
                color=f"C{idx}",
                label=f"{case_a} x {case_b}",
                edgecolor="k",
                alpha=0.5,
            ),
        )
    (ln_uq_cr,) = (
        axes.bar(
            time_step + offset,
            n_reject_uq_cr,
            width=bar_width / 4,
            color=f"C{idx}",
            label=f"{case_a} x {case_b} [corr]",
            edgecolor="k",
            alpha=0.9,
        ),
    )
    axes.bar_label(ln_uq_cr, padding=3, color=ln_uq_cr[-1].get_facecolor())

    # ln_lq, = axes[0].plot(n_reject_lq, color="darkblue", lw=1.0, ls="-", label=f"{quantile}%"),
    # ln_uq, = axes[0].plot(n_reject_uq, color="darkred", lw=1.0, ls="-", label=f"{100 - quantile}%")
    # axes.fill_between(time_step, n_reject_lq, n_reject_uq, color=ln_uq[0].get_color(), alpha=0.4)#, label=f"{100 - quantile}% CI")

axes.axhline(
    ALPHA * ks_pval.shape[1],
    color="#343",
    ls="-.",
    label=f"{ALPHA * 100} % of variables",
)
axes.set_title(f"Number of variables rejected at {(1 - ALPHA) * 100}% confidence")
axes.set_xlabel("Timestep")
axes.set_ylabel("N variables")

axes.legend()

_reject = f"{ALPHA:.2f}".replace(".", "p")
plt.tight_layout()
plt.savefig(f"plt_all_cases_new.png")

fig, axis = plt.subplots(1, 1)

_ = axis.hist(
    # [n_reject_cr[(case_a, case_a)][:, -1], n_reject_cr[(case_a, case_b)][:, -1]],
    [n_reject_cr[_case][:, -1] for _case in n_reject_cr],
    # label=[f"{case_a} x {case_a} [ctl]", f"{case_a} x {case_b} [prt]"],
    label=[f"{_ca} x {_cb}" for (_ca, _cb) in n_reject_cr],
    edgecolor="k",
    bins=np.arange(0, 30, 2),
)

axis.axvline(
    ALPHA * ks_pval.shape[1],
    color="#343",
    ls="-.",
    label=f"{ALPHA * 100}% of variables",
)
axis.set_xlabel(f"Number of rejected (p < {ALPHA}) variables")
axis.set_ylabel(
    f"Frequency (of {n_reject[(case_a, case_a)][:, -1].shape[0]} iterations)"
)
_ = plt.legend()

In [ ]:
styles = [".-", "x--"]
colors = ["C1", "C4"]
marker_size = 7
tests = {}
_pcts = {
    # "GW orog": [1, 5, 10, 20, 30, 40, 50],
    "GW orog": [1, 5, 10, 20, 30, 40, 50],
    # "GW orog": [1, 10, 20, 30],
    "clubb_c1": [1, 3, 5, 10],
}
for _param in _pcts:
    tests[_param] = [("Control", f"{_param} {_pct:.1f}%") for _pct in _pcts[_param]]
# print(tests)

In [ ]:
fig_nvar, axis_nvar = plt.subplots(1, 1, figsize=(2 * fig_width, fig_width ), dpi=fig_dpi)
fig_ntest, axis_ntest = plt.subplots(1, 1, figsize=(2 * fig_width, fig_width ), dpi=fig_dpi)
style_common = {"linewidth": 2.0, "markersize": 5}
tick_fontsize = 10
label_fontsize = 12
_quantile = ALPHA * 100.

for idx, param in enumerate(tests):
    # Plot the upper-quantile
    _plotter_nvar = axis_nvar.plot
    _plotter_ntest = axis_ntest.plot

    n_tests = {
        "Un-corrected": [
            (n_reject[_case] > REJECT_UC_THR).sum(axis=0)[-1] # / n_reject[_case].shape[0]
            for _case in tests[param]
        ],
        "Corrected": [
            (n_reject_cr[_case] > 0).sum(axis=0)[-1] # / n_reject_cr[_case].shape[0]
            for _case in tests[param]
        ],
        "Corrected [BH]": [
            (n_reject_cr_i[_case] > 0).sum(axis=0)[-1] # / n_reject_cr[_case].shape[0]
            for _case in tests[param]
        ],
    }

    _plotter_ntest(
        _pcts[param],
        n_tests["Un-corrected"],
        # styles[idx],
        "-",
        color=colors[idx],
        marker="x",
        label=f"Uncorrected {param}",
        **style_common,
    )
    _plotter_ntest(
        _pcts[param],
        n_tests["Corrected"],
        # styles[idx],
        "--",
        color=colors[idx],
        marker="o",
        label=f"Corrected {param} [BY]",
        **style_common,
    )
    _plotter_ntest(
        _pcts[param],
        n_tests["Corrected [BH]"],
        # styles[idx],
        "-.",
        color=colors[idx],
        marker="*",
        label=f"Corrected {param} [BH]",
        **style_common,
    )
    # Plot the median
    # _plotter_nvar(
    #     _pcts[param],
    #     [np.percentile(n_reject_cr[_case], 50, axis=0)[-1] for _case in tests[param]],
    #     styles[idx],
    #     color="k",
    #     label=f"{param} Median",
    # )

    lqs_cr = [
        np.percentile(n_reject_cr[_case], _quantile, axis=0)[-1]
        for _case in tests[param]
    ]
    lqs_uc = [
        np.percentile(n_reject[_case], _quantile, axis=0)[-1] for _case in tests[param]
    ]

    uqs_cr = [
        np.percentile(n_reject_cr[_case], 100 - _quantile, axis=0)[-1]
        for _case in tests[param]
    ]
    uqs_uc = [
        np.percentile(n_reject[_case], 100 - _quantile, axis=0)[-1]
        for _case in tests[param]
    ]

    _plotter_nvar(
        _pcts[param],
        np.array(uqs_cr),
        # styles[idx],
        "-",
        marker="o",
        color=colors[idx],
        label=f"{param} corrected {_quantile}%",
        **style_common,
    )
    if False:
        axis_nvar.fill_between(
            _pcts[param],
            lqs_cr,
            uqs_cr,
            alpha=0.2,
            color=colors[idx],
        )

        axis_nvar.fill_between(
            _pcts[param],
            lqs_uc,
            uqs_uc,
            alpha=0.2,
            color=colors[idx],
        )

    _plotter_nvar(
        _pcts[param],
        np.array(uqs_uc),
        # styles[idx],
        "--",
        marker="x",
        color=colors[idx],
        label=f"{param} un-corrected {_quantile}%",
        **style_common,
    )

# Put a horizontal line for the test failure threshold
axis_nvar.axhline(
    1, color="k", ls="-", lw=style_common["linewidth"], label="Global test failure threshold [cor]"
)
axis_nvar.axhline(
    REJECT_UC_THR, color="k", ls="--", lw=style_common["linewidth"], label="Global test failure threshold [unc]"
)

axis_nvar.set_xlabel("Parameter Change [%]", fontsize=label_fontsize)
axis_nvar.set_ylabel(f"Number of variables\nrejected [p < {ALPHA:.2f}]", fontsize=label_fontsize)
axis_nvar.legend( fontsize=label_fontsize)

axis_ntest.axhline(ALPHA * ks_pval.shape[0], color="k", ls="--", lw=style_common["linewidth"], label=f"{ALPHA * 100}% of tests")
# axis_ntest.axhline(ALPHA, color="k", ls="--", lw=1, label=f"{ALPHA * 100}% of tests")

axis_ntest.set_xlabel("Parameter Change [%]", fontsize=label_fontsize)
axis_ntest.set_ylabel(f"Tests with global\nsignificance [p < {ALPHA:.2f}]", fontsize=label_fontsize)
axis_ntest.legend( fontsize=label_fontsize)

for _ax in [axis_ntest.xaxis, axis_ntest.yaxis]:
    _ax.set_major_formatter(ScalarFormatter(useOffset=True))

for _ax in [axis_ntest, axis_nvar]:
    _ax.grid(visible=True, ls="--")
    sns.despine(ax=_ax, offset=10)
    _ax.tick_params(labelsize=tick_fontsize)

fig_nvar.tight_layout()
fig_ntest.tight_layout()

fig_nvar.savefig(f"plt_n_failed_vars_by_param.{fig_extn}")
fig_ntest.savefig(f"plt_n_failed_tests_by_param.{fig_extn}")

In [ ]:
test = {_key: (n_reject[_key] > REJECT_UC_THR).sum(axis=0) for _key in reject_test}
test_cr = {
    # _key: (n_reject_cr[_key] > ALPHA * ks_pval.shape[1]).sum(axis=0)
    _key: (n_reject_cr[_key] > 0).sum(axis=0)
    for _key in reject_test
}
fig, axes = plt.subplots(1, 1, figsize=(12, 5))

for idx, _case in enumerate(test):
    axes.plot(test[_case], color=f"C{idx}", label=f"{_case[0]} x {_case[1]}", lw=2.3)
    axes.plot(
        test_cr[_case],
        color=f"C{idx}",
        ls="--",
        label=f"{_case[0]} x {_case[1]} [corr]",
        lw=1.0,
    )
axes.set_ylabel(f"Number of iterations")
axes.set_xlabel("Time step")
axes.set_title("")
axes.legend()

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(12, 5))

# for idx, _case in enumerate(test):
# axes.plot(test[_case], color=f"C{idx}", label=f"{_case[0]} x {_case[1]}", lw=2.3)

styles = ["-", "--"]
for _thr in range(0, int(ALPHA * ks_pval.shape[1]) + 2):
    _test_cr = {_key: (n_reject_cr[_key] > _thr).sum(axis=0) for _key in reject_test}
    for idx, param in enumerate(tests):
        axes.plot(
            _pcts[param],
            [_test_cr[_case][-1] for _case in tests[param]],
            "o",
            ls=styles[idx % 2],
            label=f"{param} {_thr}",
        )
axes.axhline(
    500 * ALPHA,
    color="k",
    ls=styles[idx],
    label=f"{100 * ALPHA}% of bootstrap iterations",
)
axes.set_ylabel(f"Number of iterations failing")
axes.set_xlabel("Percent Change")
axes.legend()

In [ ]:
fig, axis = plt.subplots(1, 1, figsize=(12, 5))
_ = plt.boxplot(
    [n_reject_cr[_key][:, -1] for _key in n_reject_cr], vert=False, notch=True, whis=2.0
)
_ticks = np.arange(1, len(n_reject_cr.keys()) + 1)
_ = axis.set_yticks(_ticks, [_key[1] for _key in n_reject_cr])
_ = axis.set_xlabel(f"Number of rejected variables [p < {ALPHA}]")

In [ ]:
# fig, axes = plt.subplots(len(files), 2, figsize=(10, 2.5 * len(files)), sharex=True)
bins = np.arange(0, 121, 1)
ex_val = 500 / ((bins.shape[0] - 1) / 30)
fig, axis = plt.subplots(1, 1, figsize=(12, 5))
for file_ix, _file in enumerate(files):
    # case_a, case_b = _file.stem.split("_")[2:4]
    # cases = [case_a, case_b]
    case_a, case_b = cases[file_ix]
    n_iter = int(_file.stem.split("_")[-1][1:])
    ks_res = xr.open_dataset(_file)
    rmsd = np.array(
        [
            np.sqrt(
                np.mean(
                    (
                        ex_val
                        - np.histogram(ks_res.rnd_idx[_idx].values.flatten(), bins)[0]
                    )
                    ** 2
                )
            )
            for _idx in [0, 1]
        ]
    )
    # print(f"{case_a} x {case_b:25s} {rmsd[0]:.2f} x {rmsd[1]:.2f}")
    axis.plot([file_ix] * 2, rmsd, "x", label=case_b)
    # ks_res.rnd_idx[case_a]
    # for _case in [0, 1]:
    #     axes[file_ix, _case].axhline(ex_val, color="k", ls="--")
    #     _ = ks_res.rnd_idx[_case].plot.hist(
    #         edgecolor="k", bins=bins, ax=axes[file_ix, _case]
    #     )
    #     axes[file_ix, _case].set_title(cases[file_ix][_case])
    #     axes[file_ix, _case].set_xlabel("")
fig.subplots_adjust(right=0.85)
fig.suptitle("RMSD of ensemble bootstrap selection")
plt.legend(bbox_to_anchor=(0.8, 1.0, 0.5, 0.0))

In [ ]:
fig, axes = plt.subplots(len(files), 1, figsize=(10, len(files) * 2))  # , sharex=True)
bins = np.arange(0, 121, 4)
ex_val = 500 / ((bins.shape[0] - 1) / 30)

for file_ix, _file in enumerate(files):
    case_a, case_b = cases[file_ix]
    # case_a, case_b = _file.stem.split("_")[2:4]
    case_a = fmt_case(case_a)
    case_b = fmt_case(case_b)
    # cases = [case_a, case_b]
    n_iter = int(_file.stem.split("_")[-1][1:])
    ks_res = xr.open_dataset(_file)
    ks_pval = ks_res.pval
    reject_by_var = pd.DataFrame((ks_pval < ALPHA).sum(axis=0).T, columns=ks_res.vars)
    mask = reject_by_var.sum() > reject_by_var.sum().quantile(0.9)

    # reject_by_var.loc[:, reject_by_var.mean() >= 0.6].plot(ax=axes[file_ix], legend=False)
    reject_by_var.index.name = "Time Step"
    sns.heatmap(reject_by_var.T[mask], ax=axes[file_ix], vmin=0, vmax=500)
    axes[file_ix].set_title(f"{case_a} x {case_b}")
fig.tight_layout()

In [ ]:
case_a, case_b = cases[file_ix]
# case_a, case_b = _file.stem.split("_")[2:4]
case_a = fmt_case(case_a)
case_b = fmt_case(case_b)
# cases = [case_a, case_b]
n_iter = int(_file.stem.split("_")[-1][1:])
ks_res = xr.open_dataset(_file)
ks_pval = ks_res.pval
reject_by_var = pd.DataFrame((ks_pval < ALPHA).sum(axis=0).T, columns=ks_res.vars)

In [ ]:
mask = reject_by_var.sum() > reject_by_var.sum().quantile(0.8)
reject_by_var.index.name = "Time Step"
reject_by_var.T[mask].sort_values(2, ascending=False)

In [ ]:
fig, axis = plt.subplots(1, 1)

_, _, _bars = ks_res.vars[ks_res.pval.isel(time=2).argmin(dim="vars")].plot.hist(
    ax=axis
)
_ = axis.bar_label(_bars, padding=3, color="k", zorder=10)

In [ ]:
n_reject.keys()

In [ ]:
n_bins = np.arange(0, 45)
plt.subplot(2, 1, 1)
y_ctl, x_ctl, _ = plt.hist(
    n_reject[("Control", "Control")][:, -1], bins=n_bins, edgecolor="k", cumulative=1
)
plt.subplot(2, 1, 2)
y_test, x_test, _ = plt.hist(
    n_reject[("Control", "clubb_c1 10.0%")][:, -1],
    bins=n_bins,
    edgecolor="k",
    cumulative=1,
)

In [ ]:
plt.plot(x_ctl[:-1], y_ctl / y_ctl.max(), "-", label="Ctrl")
plt.plot(x_test[:-1], y_test / y_test.max(), "-", label="Test")
plt.legend()